#Session 4 - RNN are dead, and their renewed relevance
---

[GitHub Link of Session 4 Assignments' solution](https://github.com/garima-mahato/END2/blob/main/Session4-RNN_are_dead,_and_their_renewed_relevance/Readme.md)

### Assignment: 

1) Refer to the file we wrote in the class [Link](https://colab.research.google.com/drive/1-xwX32O0WYOqcCROJnnJiSdzScPCudAM?usp=sharing): 

  > Rewrite this code, but this time remove RNN and add LSTM. 

---

## Solution for Question 1 of Assignment

## Replacing RNN with LSTM
---

#### **Results:**

**Test Loss: 0.691**

**Test Acc: 50.29%**

In [ ]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   1%|          | 836k/84.1M [00:00<00:09, 8.34MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:00<00:00, 96.4MB/s]


In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [ ]:
print(vars(train_data.examples[0]))

{'text': ['Before', 'Cujo', ',', 'there', 'was', 'Lucky', 'the', 'devil', 'dog', '.', 'In', '1978,on', 'Halloween', 'night', 'the', 'movie"Devil', 'Dog', ',', 'The', 'Hound', 'of', 'Hell', '"', 'premiered', '.', 'A', 'story', 'of', 'a', 'family', 'getting', 'a', 'new', 'puppy', '(', 'from', 'a', 'farmer', 'who', 'just', 'happen', 'to', 'be', 'in', 'the', 'neighborhood', 'selling', 'fruits', 'and', 'vegetables', ')', 'because', 'their', 'dog', 'Skipper', 'was', 'killed', '.', 'Coencidence', '?', 'Everyone', 'loves', 'the', 'new', 'dog', ',', 'but', 'there', 'is', 'something', 'strange', 'about', 'him', '.', '<', 'br', '/><br', '/>It', 'is', "n't", 'long', 'until', 'the', 'father', 'Mike', 'Barry(Richard', 'Crenna', ',', 'First', 'Blood)starts', 'to', 'notice', '.', 'His', 'wife', 'Betty(Yvette', 'Mimieux', ',', 'Where', 'The', 'Boys', 'Are', ',', 'Jackson', 'County', 'Jail', ',', 'Snowbeast)is', 'different', 'and', 'his', 'kids', 'Charlie', 'and', 'Bonnie(Ike', 'Eisenman', ',', 'Witch',

In [ ]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [ ]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [ ]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 203063), (',', 192343), ('.', 166129), ('and', 109794), ('a', 109526), ('of', 100808), ('to', 93971), ('is', 76428), ('in', 61581), ('I', 54306), ('it', 53609), ('that', 49177), ('"', 44610), ("'s", 43276), ('this', 42373), ('-', 36806), ('/><br', 35659), ('was', 35096), ('as', 30590), ('with', 30113)]


In [ ]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [ ]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [ ]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.LSTM(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, (hidden, cell_state) = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        y = self.fc(hidden.squeeze(0))
        
        return y

In [ ]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,867,049 trainable parameters


In [ ]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 24s
	Train Loss: 0.693 | Train Acc: 50.48%
	 Val. Loss: 0.693 |  Val. Acc: 50.92%
Epoch: 02 | Epoch Time: 0m 24s
	Train Loss: 0.693 | Train Acc: 50.40%
	 Val. Loss: 0.693 |  Val. Acc: 50.94%
Epoch: 03 | Epoch Time: 0m 24s
	Train Loss: 0.693 | Train Acc: 50.32%
	 Val. Loss: 0.693 |  Val. Acc: 50.85%
Epoch: 04 | Epoch Time: 0m 25s
	Train Loss: 0.693 | Train Acc: 50.50%
	 Val. Loss: 0.693 |  Val. Acc: 50.80%
Epoch: 05 | Epoch Time: 0m 24s
	Train Loss: 0.693 | Train Acc: 50.54%
	 Val. Loss: 0.693 |  Val. Acc: 50.88%


In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.691 | Test Acc: 50.29%


---

## **Improvements:**

1) Used Padded sequences for variable length inputs

2) Used Adam optimizer

**Results:**

1) **Test Loss: 0.423**

2) **Test Accuracy: 83.45%**

[Colab Code Link](https://colab.research.google.com/drive/1dCrzmZTRFfb7DbezC4CbubZR-EXney_r?usp=sharing)

In [ ]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

# import data
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

#split train data
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

# build vocabulary
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 80.5MB/s]


Number of training examples: 25000
Number of testing examples: 25000
Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000
Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True, 
    device = device)

In [ ]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]

        #pack sequence
        # lengths need to be on CPU!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        
        #unpack sequence
        #output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        
        y = self.fc(hidden.squeeze(0))
        
        return y

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, PAD_IDX)

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()

        text, text_lengths = batch.text
                
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            text, text_lengths = batch.text

            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 9s
	Train Loss: 0.665 | Train Acc: 58.93%
	 Val. Loss: 0.635 |  Val. Acc: 65.38%
Epoch: 02 | Epoch Time: 0m 8s
	Train Loss: 0.583 | Train Acc: 69.05%
	 Val. Loss: 0.537 |  Val. Acc: 72.76%
Epoch: 03 | Epoch Time: 0m 8s
	Train Loss: 0.519 | Train Acc: 73.76%
	 Val. Loss: 0.539 |  Val. Acc: 74.14%
Epoch: 04 | Epoch Time: 0m 8s
	Train Loss: 0.405 | Train Acc: 82.12%
	 Val. Loss: 0.468 |  Val. Acc: 79.13%
Epoch: 05 | Epoch Time: 0m 8s
	Train Loss: 0.347 | Train Acc: 85.20%
	 Val. Loss: 0.469 |  Val. Acc: 77.53%
Epoch: 06 | Epoch Time: 0m 8s
	Train Loss: 0.293 | Train Acc: 88.29%
	 Val. Loss: 0.439 |  Val. Acc: 80.56%
Epoch: 07 | Epoch Time: 0m 9s
	Train Loss: 0.264 | Train Acc: 89.78%
	 Val. Loss: 0.423 |  Val. Acc: 83.43%
Epoch: 08 | Epoch Time: 0m 9s
	Train Loss: 0.223 | Train Acc: 91.42%
	 Val. Loss: 0.474 |  Val. Acc: 81.06%
Epoch: 09 | Epoch Time: 0m 8s
	Train Loss: 0.168 | Train Acc: 93.73%
	 Val. Loss: 0.424 |  Val. Acc: 83.81%
Epoch: 10 | Epoch Time: 0m 8

In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.423 | Test Acc: 83.45%
